<a href="https://colab.research.google.com/github/albehma/Image3DRecostruction/blob/main/fountain_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**3D-AR Project: Alberto Mautone and Arina Prostakova**

SfM: implementation of a SURF features extractors for test images (fountain and tiso) saved in a .txt file suitable for colmap, and features matching saved in a .txt file suitable for colmap.

In [ ]:
import numpy as np
import pandas as pd

FIRST PART: Recostruction on original SURF keypoints, descriptors

In [ ]:
fountain_link = 'https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-P11/images'

In [ ]:
#Packages to use SURF
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

     |████████████████████████████████| 25.0MB 1.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 158kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


Obtaining **fountain** images from online and storing them in a list.

In [ ]:
from skimage import io
fountain_imgs = []
for i in range(11):
  if(i>10):
    link = fountain_link+'/{:03}.jpg'.format(i)+'?raw=true'
  link = fountain_link+'/0{:03}.jpg'.format(i)+'?raw=true'
  print(link) #just for debugging
  img = io.imread(link)
  fountain_imgs.append(img)

https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-P11/images/0000.jpg?raw=true
https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-P11/images/0001.jpg?raw=true
https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-P11/images/0002.jpg?raw=true
https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-P11/images/0003.jpg?raw=true
https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-P11/images/0004.jpg?raw=true
https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-P11/images/0005.jpg?raw=true
https://raw.githubusercontent.com/openMVG/SfM_quality_evaluation/master/Benchmarking_Camera_Calibration_2008/fountain-

In [ ]:
print(len(fountain_imgs)) #just for debugging

11


In [ ]:
#Getting a SURF descriptor
import cv2
surf = cv2.xfeatures2d.SURF_create() #for fountain dataset, we don't use any threshold, because they are only 11 images
surf.setExtended(True)

from google.colab import files

Obtaining the .txt files for each SURF features in the format requested by Colmap

In [ ]:
for k in range(len(fountain_imgs)):
  fontana = fountain_imgs[k]
  keypoints_surf, descriptors = surf.detectAndCompute(fontana, None)
  if(k>=10):
    file_name = "00" + str(k) + ".jpg" + ".txt"
  else:
    file_name = "000" + str(k) + ".jpg" + ".txt"
  File_object = open(file_name, "w")
  File_object.write(str(descriptors.shape[0]) + " " + str(descriptors.shape[1]) + "\n")
  for i in range(descriptors.shape[0]):
    File_object.write(str(keypoints_surf[i].pt[0]) +  " " + str(keypoints_surf[i].pt[1]) + " " + str(keypoints_surf[i].size) + " " + str(keypoints_surf[i].angle) + " ")
    for j in range(descriptors.shape[1]):
      if (j==descriptors.shape[1]-1):
        File_object.write("0")
      else:
        File_object.write("0 ")
    File_object.write("\n")
  File_object.close()
  File_object = open(file_name, "r")
  File_object.read()
  files.download(file_name)

Obtaining the .txt file of the matchings as requested by Colmap

In [ ]:
bf = cv2.BFMatcher()
m_file_name = "mmm.txt"
File_object = open(m_file_name, "w")
for k in range(len(fountain_imgs)-1):
  fontana = fountain_imgs[k]
  for i in range(k+1, len(fountain_imgs)):
    fontana_next = fountain_imgs[i]
    keypoints_surf, descriptors = surf.detectAndCompute(fontana, None)
    keypoints_surf_next, descriptors_next = surf.detectAndCompute(fontana_next, None)
    if(i>=10):
      File_object.write("000" + str(k) + ".jpg" + " 00" + str(i) + ".jpg" + "\n")
    else:
      File_object.write("000" + str(k) + ".jpg" + " 000" + str(i) + ".jpg" + "\n")
    matches = bf.knnMatch(descriptors,descriptors_next,k=2)
    good = []
    for m,n in matches:
      if m.distance < 0.75*n.distance:
        good.append(m)
    for j in range(len(good)):
      File_object.write(str(good[j].queryIdx) + " " + str(good[j].trainIdx)+ "\n")
    File_object.write("\n")
File_object.close()
File_object = open(m_file_name, "r")
File_object.read()
files.download(m_file_name)

Now the original images analysis has finished: we can proceed with the reconstruction of the fountain and tiso with the original descriptors on Colmap. (See drive folder for the results)

**SECOND PART:** Let's proceed with the compression of the features and the new 3D reconstruction.

**First step:** producing the .csv files for each image to be given to the autoencoder.

In [ ]:
import csv
for k in range(len(fountain_imgs)):
  if(k<10):
    title = 'fountain_data' + str(0) + str(k) + '.csv';
  else:
    title = 'fountain_data' + str(k) + '.csv';
  with open(title, mode='w') as td:
    t_writer = csv.writer(td, delimiter=',')
    fountain = fountain_imgs[k]
    keypoints_surf, descriptors = surf.detectAndCompute(fountain, None)
    for d in descriptors:
      t_writer.writerow(d)
  files.download(title) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Uploading the most accurate model
from keras.models import load_model
ae100 = load_model('autoencoder_model_100_noreg.h5')

**Second step:** data prepapration. We include all the .csv files in a list, as we did for the original images at the beginning.

In [ ]:
csvs = []
csvs.append(pd.read_csv('/content/fountain_data00.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data01.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data02.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data03.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data04.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data05.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data06.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data07.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data08.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data09.csv', header=None))
csvs.append(pd.read_csv('/content/fountain_data10.csv', header=None))

In [ ]:
print(len(csvs)) #just for debugging

11


**Third step:** the last step consist in creating the matching with the compressed descriptors. This is done the same way as before, with the difference that, instead of using detectandcompute(....jpg), we use predict(....csv)

In [ ]:
bf = cv2.BFMatcher()
m_file_name = "mmm.txt"
File_object = open(m_file_name, "w")
for k in range(len(csvs)-1):
  fontana = csvs[k]
  for i in range(k+1, len(csvs)):
    fontana_next = csvs[i]
    descriptors = ae100.predict(fontana)
    descriptors_next = ae100.predict(fontana_next)
    if(i>=10):
      File_object.write("000" + str(k) + ".jpg" + " 00" + str(i) + ".jpg" + "\n")
    else:
      File_object.write("000" + str(k) + ".jpg" + " 000" + str(i) + ".jpg" + "\n")
    matches = bf.knnMatch(descriptors,descriptors_next,k=2)
    good = []
    for m,n in matches:
      if m.distance < 0.75*n.distance:
        good.append(m)
    for j in range(len(good)):
      File_object.write(str(good[j].queryIdx) + " " + str(good[j].trainIdx)+ "\n")
    File_object.write("\n")
File_object.close()
File_object = open(m_file_name, "r")
File_object.read()
files.download(m_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>